In [2]:
import ee

import geemap

In [3]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

# Functions

In [4]:
# Function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask) \
        .divide(10000) \
        .copyProperties(image, ['system:time_start'])  # this guy is important!

# Mask out water
def maskWater(image):
    return image.updateMask(waterMask.select('water_mask').lt(1))

# Function to filter images by Snow Probability
def maskS2_SNWPRB(image):

    mask = image.select('MSK_SNWPRB').lt(0.009) \
    
    return image.updateMask(mask) \
            .copyProperties(image, ['system:time_start'])  # this guy is important!

# Function to filter images by NDSI < 0.42
def maskS2snow(image):

    ndsi = image.normalizedDifference(['B3', 'B11']).rename('NDSI')
    image = image.addBands(ndsi)

    mask = image.select('NDSI').gt(0.42)

    return image.updateMask(mask) \
            .copyProperties(image, ['system:time_start'])

# Make an NDVI band
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi).copyProperties(image, ['system:time_start'])


# Function to get yearly statistics for the chosen index
def annual_images(y):
    range_year = ee.Filter.calendarRange(y, y, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(lambda image: image.addBands(image.metadata('system:time_start').divide(3.154e10)))) # Needed for linear regression 
    
    # Print out the number of images in the ImageCollection for each year
    num_images = filtered_dataset.size()
    
    
    # Choose the reducer based on the analysis choice
    if analysis == 'mean':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        )
    elif analysis == 'min' or analysis == 'max':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.minMax(),
            sharedInputs=True
        )
    elif analysis == 'median':
        reducer = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.median(),
            sharedInputs=True
        )

    # Use the combined reducer to get the statistics
    stats = filtered_dataset.reduce(reducer)
    return stats.set('year', y).set('num', num_images)


def clp(image):
    '''Clips a single Image to a region of interest'''
    return image.clip(aoi)

def createTimeBand(image):   
    return image.addBands(image.metadata('system:time_start').divide(3.154e10))

In [8]:
# Make an EVI band
def addEVI(image):
    evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2'),
    },
    ).rename('EVI')

    return image.addBands(evi).copyProperties(image, ['system:time_start'])



In [9]:
# GNDVI = (NIR - Green)/(NIR + Green)

def addGNDVI(image):
    gndvi = image.normalizedDifference(['B8', 'B3']).rename('GNDVI')
    return image.addBands(gndvi).copyProperties(image, ['system:time_start'])


# Build collection

In [6]:
HYBAS_ID = 3100180240

aoi = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_10").filter(ee.Filter.eq('HYBAS_ID', HYBAS_ID))


for seasonal (one year) trend, make year range one year

In [10]:
# Get water mask
waterMask = (
    ee.ImageCollection('MODIS/006/MOD44W') 
    .filter(ee.Filter.date('2015-01-01', '2015-01-02')) 
    .select('water_mask') \
    .first()
)
# Get Sentinel 2 harmonized images
dataset = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                #   // Sentinel 2 harmonized data only available for certain time
                  .filter(ee.Filter.calendarRange(2019,2023,'year'))
                #   // Filter by month. Be mindful of snow! 
                  .filter(ee.Filter.calendarRange(6,8,'month'))
                #   // Pre-filter to get less cloudy granules.
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                  .filterBounds(aoi)
                #   // This one's Toolik
                #   // .filterBounds(ee.Geometry.Point(-149.5427, 68.6267).buffer(1000))
                #   // This one's Russian tree tracks
                #   // .filterBounds(ee.Geometry.Point(133.16008, 66.82386).buffer(1000))
                  .map(clp)
                  .map(maskS2clouds)
                  .map(maskS2snow)
                  .map(maskWater)
                  .map(addNDVI)
                  .map(addEVI)
                  .map(addGNDVI)
                  .map(createTimeBand)
)

# Do analysis

In [11]:
# Pick your reducer
analysis = 'max'  # Choose 'mean', 'median', 'min', or 'max' for analysis

# Pick your index 
index_collection = dataset.select('NDVI') # NDVI, EVI, GNDVI


ANNUAL

In [10]:
# In case you want to limit beyond your ImageCollection
start_year = 2019
end_year = 2023
start_month = 7
end_month = 7

# Generate list of years
years = ee.List.sequence(start_year, end_year)

# Map over years to get yearly statistics
yearwise_ndvi = years.map(annual_images)


for item in yearwise_ndvi.getInfo():
    print("Year:", item['properties']['year'], "Number of images:", item['properties']['num'])

yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

# Get linear fit to pixelwise trend of annual max NDVI, EVI, or GNDVI
trend = yearCompCol.select(['system:time_start_mean',
                            'NDVI_max'
                             ]).reduce(ee.Reducer.linearFit())

Year: 2019 Number of images: 1
Year: 2020 Number of images: 2
Year: 2021 Number of images: 3
Year: 2022 Number of images: 4
Year: 2023 Number of images: 4


SEASONAL

In [14]:
trend = dataset.select(['system:time_start_mean',
                            'NDVI_max'
                             ]).reduce(ee.Reducer.linearFit())


map


In [12]:
long = aoi.geometry().centroid().coordinates().get(0).getInfo()
lat = aoi.geometry().centroid().coordinates().get(1).getInfo()

In [13]:
Map = geemap.Map(center = (lat, long), zoom = 11)

Map.addLayer(dataset, {
    'min':0.0,
    'max':0.3,
    'bands': ['B4', 'B3', 'B2']}, 'RGB')

Map.addLayer(trend.select('scale'),
              {'min':-0.5, 'max':0.5,
            'palette': ['red', 'white', 'blue']},
 'trend')


Map.add_colorbar_branca(colors=['red', 'white', 'blue'], vmin=-0.5, vmax=0.5, layer_name='trend')

Map

Map(center=[66.85788224579, 134.7459434786377], controls=(WidgetControl(options=['position', 'transparent_bg']…

In [11]:
index_collection = dataset.select('NDVI') 

def get_ndvi_trend(start_yr, end_yr, start_m, end_m, ndvi_stat): # ndvi_stat = 'min' 'mean' etc
    
    analysis = ndvi_stat
    start_year = start_yr
    end_year = end_yr
    start_month = start_m
    end_month = end_m
    
    years = ee.List.sequence(start_year, end_year)
    yearwise_ndvi = years.map(annual_images)

    yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

    trend = yearCompCol.select(['system:time_start_mean', 'ndvi_'+analysis]).reduce(ee.Reducer.linearFit())

    vis_params = {'min': -0.1, 'max': 0.1, 'palette': ['red', 'white', 'blue']}
    
    image = trend.select('scale').visualize(**vis_params)

    return image

In [24]:
trend1 = get_ndvi_trend(2019, 2023, 6, 8, 'ndvi_min')


NameError: name 'start_month' is not defined

In [20]:
# Initialize an empty image collection
trend_collection = ee.ImageCollection([])

# Add each trend image to the collection
trend_collection = trend_collection.merge(trend.select(['scale']))

# Visualize the trend collection
vis_params = {'min': -0.1, 'max': 0.1, 'palette': ['red', 'white', 'blue']}
Map.addLayer(trend_collection, vis_params, 'Trend Collection')


NameError: name 'trend' is not defined

In [ ]:
from matplotlib import gridspec
import matplotlib as mpl
import numpy as np
# compute the number of rows and columns

n_plots = 4
n_cols = 2
n_rows = 2
# setup the plot
scale = max(n_cols, n_rows)
fig = plt.figure(figsize=(5* scale, 5* scale))
grid = gridspec.GridSpec(n_rows, n_cols, fig, wspace=0.4)
# iterate through each subplot and plot each image
for i in range(n_plots):
    ax = fig.add_subplot(grid[i])
    show(images[i]['rgb'],
                        transform=transform_window, 
                        ax=ax,  
                        alpha=.65, 
                        title = images[i]['date'] 
                       )
